<a href="https://colab.research.google.com/github/dvillasxUdg/ti_proyecto_final/blob/main/Proyecto_Final_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module and Init values

In [1]:
pip install pybaselines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 3.1 MB/s eta 0:00:00


In [2]:
import json
import numpy as np
import panel as pn
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from pybaselines import Baseline, utils

import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

with open('average_coherence_data.json', 'r') as json_file:
    loaded_coherence_data = json.load(json_file)

participants = [
    'ADJRR', 'CESARMDG', 'EMR', 'HMM', 'JAVI', 'JSDR', 'LESR', 'MEGA', 'VMAP',
    'ABN', 'AGM', 'AJJDT', 'AMCJ', 'AMGP', 'IGNR', 'MFGM', 'MJLS', 'NYHG', 'SDAMG'
]
WORD_6 = '6'
WORD_7 = '7'
BAND = 'delta'
CHANNELS = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'Fz', 'Cz', 'Pz']

# Functions

In [3]:
# Plot Coherences values
def plot_coherence(coherence_matrix):
    coherence_matrix = coherence_matrix[::-1]

    trace = go.Heatmap(z=coherence_matrix,
                       x=CHANNELS,
                       y=CHANNELS[::-1],
                       colorscale='Viridis', zmin=0, zmax=1)

    layout = go.Layout(
        xaxis=dict(showline=True, showgrid=False, tickvals=list(range(len(CHANNELS))), ticktext=CHANNELS),
        yaxis=dict(showline=True, showgrid=False, tickvals=list(range(len(CHANNELS))), ticktext=CHANNELS[::-1]),
        autosize=False,
        width=400,
        height=400,
    )

    fig = go.Figure(data=[trace], layout=layout)
    return fig

# Plot Step length
def plot_histo(x):
  fig = go.Figure()
  fig.add_trace(go.Histogram(
      x=x,
      xbins=dict(
        size=0.01
    )))
  return fig

# Update panels
def update_panels(event):
    # Widgets values
    metric = metric_selector.value
    participant_name = participants_selector.value

    # Coherence values
    coherence_6 = loaded_coherence_data[participant_name][WORD_6][BAND]
    coherence_7 = loaded_coherence_data[participant_name][WORD_7][BAND]

    if metric == 'Coherencia':
      fig_cohe_6 = plot_coherence(coherence_6)
      fig_cohe_7 = plot_coherence(coherence_7)
    else:
      cohe_values_6 = np.asarray(coherence_6).reshape(-1)
      cohe_values_7 = np.asarray(coherence_7).reshape(-1)
      fig_cohe_6 = plot_histo(cohe_values_6)
      fig_cohe_7 = plot_histo(cohe_values_7)


    fig_combined = make_subplots(rows=1, cols=2, subplot_titles=[f'Congruente palabra {WORD_6}', f'Incongruente palabra {WORD_7}'])
    for trace in fig_cohe_6.data:
      fig_combined.add_trace(trace, row=1, col=1)
    for trace in fig_cohe_7.data:
      fig_combined.add_trace(trace, row=1, col=2)


    if metric != 'Coherencia':
      fig_combined.update_layout(
        xaxis_title_text='Value', # xaxis label
        yaxis_title_text='Count', # yaxis label
        )

    panel.object = fig_combined



# Main

In [4]:
# Widgets
participants_selector = pn.widgets.Select(name='Participante', options=participants)
metric_selector = pn.widgets.Select(name='Metrica', options=['Coherencia', 'Step Length'])

# Watcher
participants_selector.param.watch(update_panels, 'value')
metric_selector.param.watch(update_panels, 'value')

# Init values
participant_name = participants_selector.value
coherence_6 = loaded_coherence_data[participant_name][WORD_6][BAND]
coherence_7 = loaded_coherence_data[participant_name][WORD_7][BAND]

initial_fig_combined = make_subplots(rows=1, cols=2, subplot_titles=[f'Congruente palabra {WORD_6}', f'Incongruente palabra {WORD_7}'])

for fig, col in [(plot_coherence(coherence_6), 1),
                 (plot_coherence(coherence_7), 2)]:
    for trace in fig.data:
        initial_fig_combined.add_trace(trace, row=1, col=col)

panel = pn.pane.Plotly(initial_fig_combined)

# Set dashboard
layout = pn.Row(
    pn.Column(
        participants_selector,
        metric_selector
    ),
    pn.Column(
        panel,
        width=1000
    )
)

layout.servable()


Row
    [0] Column
        [0] Select(name='Participante', options=['ADJRR', 'CESARMDG', ...], value='ADJRR')
        [1] Select(name='Metrica', options=['Coherencia', ...], value='Coherencia')
    [1] Column(width=1000)
        [0] Plotly(Figure)

# Original Plot and Base Line

In [6]:
with open('process_data_origin.json', 'r') as json_file:
    origin_data = json.load(json_file)

In [17]:
from plotly.graph_objects import YBins
from numpy.core.multiarray import asanyarray
def plot_2d_eeg(fig, matrix):
  x = np.linspace(0, 700, 175)

  for i in range(len(matrix)):
    fig.add_trace(go.Scatter(
        x = x,
        y = matrix[i],
        marker = dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = CHANNELS[i],
        showlegend = True
    ))

  return fig

def make_average(data):
  x = data.sum(axis=0)/34
  return x

def make_base_line(matrix_array):
  matrix_basel_line = []
  x = np.linspace(0, 700, 175)

  # print(matrix_array.shape)

  for matrix in matrix_array:
    word_matrix_base_line = []

    # print(matrix.shape)

    for i in range(len(matrix)):
      baseline_fitter = Baseline(x_data=x)
      base_line = baseline_fitter.fabc(matrix[i])
      array_base_line = np.asarray(base_line[0])
      word_matrix_base_line.append(np.subtract(matrix[i], array_base_line))

  matrix_basel_line.append(word_matrix_base_line)

  return np.asanyarray(matrix_basel_line)

@pn.depends(participants_selector)
def data_plot(participants_selector):
  fig_6 = go.Figure()
  matrix_avg = make_average(np.asanyarray(origin_data[participants_selector]['6']))
  fig_6 = plot_2d_eeg(fig_6, matrix_avg)

  fig_7 = go.Figure()
  matrix_base_line = make_base_line(np.asanyarray(origin_data[participants_selector]['7']))
  matrix_avg = make_average(matrix_base_line)
  fig_7 = plot_2d_eeg(fig_7, matrix_avg)

  return pn.Row(fig_6, fig_7)

In [18]:
pn.Row(participants_selector, data_plot)


Row
    [0] Select(name='Participante', options=['ADJRR', 'CESARMDG', ...], value='CESARMDG')
    [1] ParamFunction(function, _pane=Row, defer_load=False)